# 阶梯三 远程文件安全传输
## 题目
1. 真实网络环境实现任意类型文件的远程传输
2. 网络传输采用socket或SSL，二选一
3. 支持任意类型的文件
4. 能够实现文件的正确加解密
## 思路
1. 双方建立连接——用Socket类进行编程
2. 双方达成一致密钥——使用RSA生成Server的公钥私钥对
3. 发送方使用密钥对发送文件加密
4. 接收方使用密钥对接收文件解密
5. 通话完毕，关闭连接
## 注意事项
1. 展示加密结果


In [6]:
#!coding=utf-8
 
import socket
import os
import sys
import struct
#import pyDes
import base64
from Crypto.Cipher import AES
import time
import rsa

key0 = '6A4B3C7D9E2F1F3F'
vi = '0102030405060708'
BUFF = 1024

def pkcs7padding(text):
        """
        明文使用PKCS7填充
        最终调用AES加密方法时，传入的是一个byte数组，要求是16的整数倍，因此需要对明文进行处理
        :param text: 待加密内容(明文)
        :return:
        """
        bs = AES.block_size  # 16
        length = len(text)
        bytes_length = len(bytes(text, encoding='utf-8'))
        # tips：utf-8编码时，英文占1个byte，而中文占3个byte
        padding_size = length if(bytes_length == length) else bytes_length
        padding = bs - padding_size % bs
        # tips：chr(padding)看与其它语言的约定，有的会使用'\0'
        padding_text = chr(padding) * padding
        return text + padding_text

def socket_client():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect(('localhost', 9001))
    except socket.error as msg:
        print (msg)
        sys.exit(1)
    print (s.recv(1024))
    
    # 加上给密钥加上时间戳，保证相同的明文，每次发送的密文不同
    t = time.time()
    t_str = str(int(t))
    key = key0[0:8] + t_str[2:10]                   # 取时间戳中的后八位
    print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+' start to change key!')
    #客户端发送‘changekey’开始进行密钥交换
    s.send('changekey'.encode('utf-8'))
    #客户端收到公钥
    modulus = int(s.recv(BUFF).decode('utf-8'))
    exponent = int(s.recv(BUFF).decode('utf-8'))
    
    #客户端使用公钥对AES密钥进行加密
    print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+' start to build pubkey')
    pubkey = rsa.PublicKey(modulus, exponent)
    
    print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+' encrypt aes-key')
    crypto = rsa.encrypt(key.encode('utf-8'), pubkey)
    
    #将加密后的AES密钥发送给服务端
    print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))+' send encrypted aes-key:'+crypto.decode('utf8','ignore'))
    s.send(crypto)

    # 需要传输的文件路径
    filepath = 'C:/Users/zhizh/Desktop/2022-spring/1.jpg'
    # 判断是否为文件
    if os.path.isfile(filepath):
        # 定义定义文件信息。128s表示文件名为128bytes长，l表示一个int或log文件类型，在此为文件大小
        fileinfo_size = struct.calcsize('128sl')
        # 定义文件头信息，包含文件名和文件大小
        fhead = struct.pack('128sl', os.path.basename(filepath).encode('utf-8'), os.stat(filepath).st_size)
        # 发送文件名称与文件大小
        s.send(fhead)
        
        
        #method=pyDes.des(key,pyDes.ECB,IV,pad=None,padmode=pyDes.PAD_PKCS5)
        # 将传输文件以二进制的形式分多次上传至服务器
        fp = open(filepath, 'rb')
        while 1:
            data = fp.read(1024)
            print(data)
            if not data:
                print ('{0} file send over...'.format(os.path.basename(filepath)))
                break
            
            #pad = lambda s: (s + (16 - len(s) % 16) * chr(16 - len(s) % 16) )  
            #newdata=pad(data)
            if len(data)<1024:
                newdata=data.decode('utf8','ignore')
                print(len(newdata))
                newdata=pkcs7padding(newdata)
                print(len(newdata))
                data=newdata.encode('utf8')
            # 字符串补位
            cipher = AES.new(key.encode('utf8'), AES.MODE_CBC, vi.encode('utf8'))
            #encryptedbytes = cipher.encrypt(newdata.encode('utf8'))
            encryptedbytes = cipher.encrypt(data)
            # 加密后得到的是bytes类型的数据
            #encodestrs = base64.b64encode(encryptedbytes)
            # 使用Base64进行编码,返回byte字符串
            #enctext = encodestrs.decode('utf8')
            
            #data=method.encrypt(data)
            s.send(encryptedbytes)
            print(encryptedbytes)
            print(len(encryptedbytes))
        # 关闭当期的套接字对象
        s.close()
        
if __name__ == '__main__':
    socket_client()

b'Hi, Welcome to the server!'
2022-04-28 19:24:25 start to change key!


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1024
b'\x01\x90\xc3?\x8dlk\x9av\xb1w\xe1\xa6\x8fNy\x18\x17\xf9\x8c\x87\x01OPs\x92s\xcf\x18\xc5|\x16s]\xd1\xc1\xcakuo\xcd\x1f\xa0d\xb8uW\x1b\nog\xf9Y\x9e\xe5k\x047N\x88\x96\x86X\xd1A\r\x11\x08\xab\xeaNO\xca\x07\x18\x079\xedS\x0b\x0f6\x17"2\xe8\xdc\t\x06\xd1\xc7nO\'\xf9\xd7\x1f\xf0\xa7Y\xd4.\xf4h\xff\x00\xb6dEr|\xa3\x12\xb8\xca\x05\xe0d\xfb\x8e{\xf6\xf5\xafQ\xbf\x9a\x08\x94\xc6\xcf\x14\x81NN~o\xc8z\xd7\xb9C\x10\xaaS\x8dH\xec\xd2\x7fz>{\x13FT\xaa\xca\x8c\xba6\xbe\xe3\x98\xb4\xbb\xd6\xa3\xbb\x86\xc5Z<oP\xa4s\xb0\x7f\xb4\xd9\xef\xde\xbe\xf6\x10\xdd\\"\x18\xa5FM\xa7\xe6%q\x8fA\xdc\xfe<W\xc6\xf0\xcd`d\xb7{\x84L1V\x08\xab\xeay\'\x1f\xa8\xaf\xaa\xc4\xcc\xcd\xe5\xe8\xd6\x85\xed\xb2L\x8d"0_|(5\xec\xe5\x94\xae\x9b<\\\xc9sr\xb3_\xec\xfa|\xb0,R\xce\xe8\x15\xb2\x02\xae\xd7\'\x9f`H\xfc\xea)|9\'\x96\x12\xdao8\x9c\xb0$ \xc7\xe6\x7f\xfa\xf5(\x99\xe3Uk\x92\x91\xa3\x8c\x85u\'h\xed\x8e8\xcf\xbf\xbd@\xcc\xa1I\xb6\x03\xcd \x9d\xcb\x93\xc7\xba\xb1\xafJNJ;\x1etSJ\xcc\x82m"kK\x8f\xf4\x83\xf3\xbfV8\x08\x00\x1e\xa